In [3]:
import pandas as pd
import networkx as nx
import numpy as np
import torch

In [4]:
data = pd.read_csv('D:\\学习\\论文\\2024new-第二篇\\pmy\\dataset\\NF-BoT-IoT\\data\\NF-BoT-IoT.csv')

In [5]:
data

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
0,192.168.100.6,52670,192.168.100.1,53,17,5.212,71,126,1,1,0,4294966,0,Benign
1,192.168.100.6,49160,192.168.100.149,4444,6,0.000,217753000,199100,4521,4049,24,4176249,1,Theft
2,192.168.100.46,3456,192.168.100.5,80,17,0.000,8508021,8918372,9086,9086,0,4175916,0,Benign
3,192.168.100.3,80,192.168.100.55,8080,6,7.000,8442138,9013406,9086,9086,0,4175916,0,Benign
4,192.168.100.46,80,192.168.100.5,80,6,7.000,8374706,0,9086,0,0,4175916,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600095,192.168.100.46,80,192.168.100.5,80,6,7.000,2330065,0,2523,0,0,4263037,0,Benign
600096,192.168.100.5,0,192.168.100.3,0,6,0.000,1054423,0,1513,0,0,4263062,0,Benign
600097,192.168.100.7,365,192.168.100.3,565,17,0.000,62422,0,1357,0,0,4263062,0,Benign
600098,192.168.100.3,50850,13.54.166.67,8883,6,222.178,11300,1664,32,32,24,4264935,0,Benign


In [9]:
"""
Gf(Vf,Ef)
Vf: ip
Ef: protocl, flow duration, incoming flow bytes, bytes per packet, TCP flags of the first packet
"""
# 提取 sourceIP 和 dstIP 的所有唯一组合
ip_combinations = data[['IPV4_SRC_ADDR', 'IPV4_DST_ADDR']].drop_duplicates()
ip_combinations


,IPV4_SRC_ADDR,IPV4_DST_ADDR
0,192.168.100.6,192.168.100.1
1,192.168.100.6,192.168.100.149
2,192.168.100.46,192.168.100.5
3,192.168.100.3,192.168.100.55
5,192.168.100.5,192.168.100.3
...,...,...
599133,192.168.100.3,205.251.199.203
599136,192.168.100.3,205.251.194.102
599186,192.168.100.3,54.200.220.170
599207,192.168.100.6,203.5.76.9


In [7]:
data = pd.read_csv('D:\\学习\\论文\\2024new-第二篇\\pmy\\dataset\\NF-BoT-IoT\\data\\NF-BoT-IoT.csv')

In [5]:
filtered_IP_list = []
# 提取 sourceIP 和 dstIP 的所有唯一组合
ip_combinations = data[['IPV4_SRC_ADDR', 'IPV4_DST_ADDR']].drop_duplicates()
# 遍历每个组合
for _, row in ip_combinations.iterrows():
    source_ip = row['IPV4_SRC_ADDR']
    dst_ip = row['IPV4_DST_ADDR']
    # 筛选出匹配的行
    filtered_data = data[(data['IPV4_SRC_ADDR'] == source_ip) & (data['IPV4_DST_ADDR'] == dst_ip)]
    # 将筛选结果存入数组（包含元信息）
    filtered_IP_list.append({
        "IPV4_SRC_ADDR": source_ip,
        "IPV4_DST_ADDR": dst_ip,
        "data": filtered_data
    })

# 找出每个 IP 组的 data 个数
data_lengths = [len(group["data"]) for group in filtered_IP_list]
max_length = max(data_lengths)

# # 在随机位置插入空值
# for group in filtered_IP_list:
#     data = group["data"]
#     current_length = len(data)
# 
#     if current_length < max_length:
#         num_rows_to_add = max_length - current_length
# 
#         # 构建空值 DataFrame，与原数据保持相同的列名
#         zero_rows = pd.DataFrame(0, index=range(num_rows_to_add), columns=data.columns)
# 
#         # # 随机插入空值
#         # insertion_indices = np.random.choice(range(current_length + num_rows_to_add), size=num_rows_to_add,
#         #                                      replace=False)
# 
#         # 指定插入位置（这里以固定位置为例，可根据需求更改）
#         insertion_indices = np.linspace(0, current_length, num=num_rows_to_add, endpoint=False, dtype=int)
# 
#         # 将原数据和全 0 行合并
#         for index in insertion_indices:
#             data = pd.concat([data.iloc[:index], zero_rows.iloc[:1], data.iloc[index:]]).reset_index(drop=True)
#             zero_rows = zero_rows.iloc[1:]  # 更新 zero_rows 剩余部分
# 
#         # 更新组的 data
#         group["data"] = data
            
            

In [6]:
# 构建时空图：Dynamic Spatiotemporal Graph Snapshots
def build_graph(data, max_edges_per_snapshot):
    """
    根据网络流的顺序生成动态图快照
    Args:
    - data: DataFrame，包含网络流量记录，字段包括：
        - IPV4_SRC_ADDR: 源IP地址
        - IPV4_DST_ADDR: 目标IP地址
        - PROTOCOL: 协议类型
        - FLOW_DURATION_MILLISECONDS: 流持续时间
        - IN_BYTES
        - OUT_BYTES
        - bytes_per_packet: 每个包的字节数 暂时设成 in+out / packetsNum
        - TCP_FLAGS: TCP标志
    - max_edges_per_snapshot: 每个快照中允许的最大边数（控制快照大小）
    
    Returns:
    - snapshots: 包含多个图快照的列表，每个快照是一个 NetworkX 图
    """
    snapshots = []
    G = nx.DiGraph()  # 初始化一个有向图
    edge_count = 0
    for _, row in data:
        src, dest = row['IPV4_SRC_ADDR'], row['IPV4_DST_ADDR']

        # 添加节点
        if not G.has_node(src):
            G.add_node(src, type='IP')
        if not G.has_node(dest):
            G.add_node(dest, type='IP')

        # 添加边及其特征
        G.add_edge(
            src,dest,
            protocol=row['PROTOCOL'],
            flow_duration=row['FLOW_DURATION_MILLISECONDS'],
            incoming_flow_bytes=row['IN_BYTES'],
            bytes_per_packet=(row['IN_BYTES']+row['OUT_BYTES'])/(row['IN_PKTS']+row['OUT_PKTS']),
            tcp_flags=row['TCP_FLAGS']
        )
        edge_count += 1

        # 当达到最大边数时，保存当前快照，开始一个新图
        if edge_count >= max_edges_per_snapshot:
            snapshots.append(G)
            G = nx.DiGraph()  # 创建新图
            edge_count = 0

    # 如果最后一个图有剩余边，也需要保存
    if G.number_of_edges() > 0:
        snapshots.append(G)

    return snapshots

    

In [7]:
'生成动态图快照'
max_edges_per_snapshot = 50  # 每个快照最多包含50条边
snapshots = build_graph(data, max_edges_per_snapshot)

# 打印每个图快照的节点和边
for i, G in enumerate(snapshots):
    print(f"Snapshot {i}:")
    print("Nodes:", G.nodes(data=True))
    print("Edges:", G.edges(data=True))
    if(i > 1):
        break

Snapshot 0:
Nodes: [('192.168.100.6', {'type': 'IP'}), ('192.168.100.1', {'type': 'IP'}), ('192.168.100.149', {'type': 'IP'}), ('192.168.100.46', {'type': 'IP'}), ('192.168.100.5', {'type': 'IP'}), ('192.168.100.3', {'type': 'IP'}), ('192.168.100.55', {'type': 'IP'}), ('192.168.100.7', {'type': 'IP'}), ('13.54.166.67', {'type': 'IP'}), ('192.168.100.150', {'type': 'IP'}), ('192.168.100.255', {'type': 'IP'}), ('27.124.125.251', {'type': 'IP'}), ('192.168.100.4', {'type': 'IP'}), ('91.189.94.4', {'type': 'IP'})]
Edges: [('192.168.100.6', '192.168.100.1', {'protocol': 17, 'flow_duration': 4294966, 'incoming_flow_bytes': 71, 'bytes_per_packet': 98.5, 'tcp_flags': 0}), ('192.168.100.6', '192.168.100.149', {'protocol': 6, 'flow_duration': 4176249, 'incoming_flow_bytes': 217753000, 'bytes_per_packet': 25431.98366394399, 'tcp_flags': 24}), ('192.168.100.6', '192.168.100.3', {'protocol': 6, 'flow_duration': 4175008, 'incoming_flow_bytes': 898452, 'bytes_per_packet': 282.0, 'tcp_flags': 0}), ('1

In [8]:
def generate_line_graph_snapshots(graph_snapshots):
    """
    基于时空图快照生成线图快照
    Args:
    - graph_snapshots: List[networkx.Graph]，原始时空图快照的列表

    Returns:
    - line_graph_snapshots: List[networkx.Graph]，线图快照的列表
    """
    line_graph_snapshots = []

    for i, G in enumerate(graph_snapshots):
        # 使用 NetworkX 的 line_graph 方法生成线图
        line_graph = nx.line_graph(G)

        # 传递原始图的边特征到线图的节点
        for edge in line_graph.nodes:
            # edge 是原始图中的一条边，例如 (u, v)
            if G.has_edge(*edge):
                # 将原始图边的属性复制到线图节点
                line_graph.nodes[edge].update(G.edges[edge])

        line_graph_snapshots.append(line_graph)

    return line_graph_snapshots

# 示例原始时空图快照（生成简单图作为示例）
def create_sample_graph_snapshots():
    G1 = nx.Graph()
    G1.add_edge('192.168.1.1', '192.168.1.2', protocol='TCP', bytes=100)
    G1.add_edge('192.168.1.2', '192.168.1.3', protocol='UDP', bytes=200)

    G2 = nx.Graph()
    G2.add_edge('192.168.1.1', '192.168.1.3', protocol='TCP', bytes=300)
    G2.add_edge('192.168.1.3', '192.168.1.4', protocol='UDP', bytes=400)

    return [G1, G2]

# # 生成原始时空图快照
# graph_snapshots = create_sample_graph_snapshots()
# 
# # 生成线图快照
# line_graph_snapshots = generate_line_graph_snapshots(graph_snapshots)
# 
# # 打印每个线图快照的节点和边
# for i, LG in enumerate(line_graph_snapshots):
#     print(f"Line Graph Snapshot {i}:")
#     print("Nodes:", LG.nodes(data=True))
#     print("Edges:", LG.edges())


In [9]:
'生成线图快照'
line_graph_snapshots = generate_line_graph_snapshots(snapshots)

In [10]:
def generate_line_graph_matrices(graph_snapshots):
    """
    生成线图的邻接矩阵和特征矩阵序列
    Args:
    - graph_snapshots: List[networkx.Graph]，原始图快照列表，每个图包含边特征

    Returns:
    - adjacency_matrices: List[torch.Tensor]，线图邻接矩阵列表 (A1, A2, ..., Ak)
    - feature_matrices: torch.Tensor，线图特征矩阵序列 (X1, X2, ..., Xk)，形状为 (k, n, d)
    """
    adjacency_matrices = []  # 存储线图的邻接矩阵
    feature_matrices = []    # 存储线图的特征矩阵

    for G in graph_snapshots:
        # Step 1: 创建线图
        line_graph = nx.line_graph(G)

        # Step 2: 获取线图邻接矩阵
        nodes = list(line_graph.nodes)  # 线图节点（对应原始图的边）
        node_to_idx = {node: idx for idx, node in enumerate(nodes)}
        n = len(nodes)
        A = np.zeros((n, n), dtype=np.float32)
        for u, v in line_graph.edges:
            A[node_to_idx[u], node_to_idx[v]] = 1.0
            A[node_to_idx[v], node_to_idx[u]] = 1.0
        adjacency_matrices.append(torch.tensor(A))

        # Step 3: 获取线图特征矩阵
        d = len(G.edges[nodes[0]])  # 原始图边的特征维度
        X = np.zeros((n, d), dtype=np.float32)
        for edge, idx in node_to_idx.items():
            X[idx] = np.array([G.edges[edge][attr] for attr in G.edges[edge]])
        feature_matrices.append(torch.tensor(X))

    # 将特征矩阵堆叠为 (k, n, d)
    feature_matrices = torch.stack(feature_matrices)

    return adjacency_matrices, feature_matrices

# 示例原始图快照
def create_sample_graph_snapshots():
    G1 = nx.Graph()
    G1.add_edge('192.168.1.1', '192.168.1.2', protocol=1, flow_duration=5, bytes=100)
    G1.add_edge('192.168.1.2', '192.168.1.3', protocol=2, flow_duration=10, bytes=200)

    G2 = nx.Graph()
    G2.add_edge('192.168.1.1', '192.168.1.3', protocol=1, flow_duration=8, bytes=150)
    G2.add_edge('192.168.1.3', '192.168.1.4', protocol=3, flow_duration=12, bytes=300)

    return [G1, G2]


In [11]:
'生成线图邻接矩阵和特征矩阵'
adjacency_matrices, feature_matrices = generate_line_graph_matrices(line_graph_snapshots)
adjacency_matrices

IndexError: list index out of range

In [6]:
import networkx as nx

def graph_to_line_graph(G_f):
    line_graph_snapshots = []
    # for G_f in graph_snapshots:
    # 创建线图
    L_f = nx.line_graph(G_f)  # 利用NetworkX生成线图
    line_graph_snapshots.append(L_f)
    return L_f

# 示例输入
G_f = nx.Graph()
G_f.add_edges_from([(1, 2), (2, 3), (3, 4),(4,5),(5,1),(4,2)])  # 添加边

# 转换为线图
line_graph_snapshots = graph_to_line_graph(G_f)
# for L_f in line_graph_snapshots:
print("线图节点:", line_graph_snapshots.nodes(data=True))
print("线图边:", line_graph_snapshots.edges(data=True))


线图节点: [((2, 4), {}), ((3, 4), {}), ((4, 5), {}), ((1, 5), {}), ((1, 2), {}), ((2, 3), {})]
线图边: [((2, 4), (3, 4), {}), ((2, 4), (4, 5), {}), ((2, 4), (2, 3), {}), ((2, 4), (1, 2), {}), ((3, 4), (2, 3), {}), ((3, 4), (4, 5), {}), ((4, 5), (1, 5), {}), ((1, 5), (1, 2), {}), ((1, 2), (2, 3), {})]


In [7]:
import numpy as np
import networkx as nx

# 创建原图
G = nx.Graph()
G.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 0)])  # 原图有4条边

# 获取原图邻接矩阵
A = nx.adjacency_matrix(G).toarray()

# 获取原图的边列表
edges = list(G.edges)
num_edges = len(edges)

# 初始化线图邻接矩阵
A_L = np.zeros((num_edges, num_edges))

# 构造线图邻接矩阵
for i, (u1, v1) in enumerate(edges):
    for j, (u2, v2) in enumerate(edges):
        # 如果两条边共享一个端点，则它们在线图中相连
        if len(set([u1, v1]).intersection([u2, v2])) > 0 and i != j:
            A_L[i, j] = 1

print("线图邻接矩阵:")
print(A_L)


线图邻接矩阵:
[[0. 1. 1. 0.]
 [1. 0. 0. 1.]
 [1. 0. 0. 1.]
 [0. 1. 1. 0.]]


In [8]:
# 假设原图的每个节点有一个特征（示例：节点编号作为特征）
node_features = {i: [i] for i in G.nodes}

# 生成线图的节点特征矩阵
X_L = []
for u, v in edges:
    # 方法1: 将端点特征拼接
    feature = node_features[u] + node_features[v]
    X_L.append(feature)

X_L = np.array(X_L)
print("线图节点特征矩阵:")
print(X_L)


线图节点特征矩阵:
[[0 1]
 [0 3]
 [1 2]
 [2 3]]


In [3]:
import pandas as pd
data = pd.read_csv('D:\\学习\\论文\\2024new-第二篇\\pmy\\dataset\\NF-BoT-IoT\\data\\NF-BoT-IoT.csv')

In [8]:
label = data['Label']
label.shape
data

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
0,192.168.100.6,52670,192.168.100.1,53,17,5.212,71,126,1,1,0,4294966,0,Benign
1,192.168.100.6,49160,192.168.100.149,4444,6,0.000,217753000,199100,4521,4049,24,4176249,1,Theft
2,192.168.100.46,3456,192.168.100.5,80,17,0.000,8508021,8918372,9086,9086,0,4175916,0,Benign
3,192.168.100.3,80,192.168.100.55,8080,6,7.000,8442138,9013406,9086,9086,0,4175916,0,Benign
4,192.168.100.46,80,192.168.100.5,80,6,7.000,8374706,0,9086,0,0,4175916,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600095,192.168.100.46,80,192.168.100.5,80,6,7.000,2330065,0,2523,0,0,4263037,0,Benign
600096,192.168.100.5,0,192.168.100.3,0,6,0.000,1054423,0,1513,0,0,4263062,0,Benign
600097,192.168.100.7,365,192.168.100.3,565,17,0.000,62422,0,1357,0,0,4263062,0,Benign
600098,192.168.100.3,50850,13.54.166.67,8883,6,222.178,11300,1664,32,32,24,4264935,0,Benign


In [9]:
from sklearn.preprocessing import StandardScaler
# 数值标准化（零均值和单位方差）
scaler = StandardScaler()
data['PROTOCOL'] = scaler.fit_transform(data[['PROTOCOL']])

In [10]:
data

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
0,192.168.100.6,52670,192.168.100.1,53,4.057229,5.212,71,126,1,1,0,4294966,0,Benign
1,192.168.100.6,49160,192.168.100.149,4444,-0.227830,0.000,217753000,199100,4521,4049,24,4176249,1,Theft
2,192.168.100.46,3456,192.168.100.5,80,4.057229,0.000,8508021,8918372,9086,9086,0,4175916,0,Benign
3,192.168.100.3,80,192.168.100.55,8080,-0.227830,7.000,8442138,9013406,9086,9086,0,4175916,0,Benign
4,192.168.100.46,80,192.168.100.5,80,-0.227830,7.000,8374706,0,9086,0,0,4175916,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600095,192.168.100.46,80,192.168.100.5,80,-0.227830,7.000,2330065,0,2523,0,0,4263037,0,Benign
600096,192.168.100.5,0,192.168.100.3,0,-0.227830,0.000,1054423,0,1513,0,0,4263062,0,Benign
600097,192.168.100.7,365,192.168.100.3,565,4.057229,0.000,62422,0,1357,0,0,4263062,0,Benign
600098,192.168.100.3,50850,13.54.166.67,8883,-0.227830,222.178,11300,1664,32,32,24,4264935,0,Benign
